In [1]:
pip install /kaggle/input/cibmtr-whl-files-for-installation/scikit_survival-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


Processing /kaggle/input/cibmtr-whl-files-for-installation/scikit_survival-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install /kaggle/input/cibmtr-whl-files-for-installation/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/cibmtr-whl-files-for-installation/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/cibmtr-whl-files-for-installation/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/cibmtr-whl-files-for-installation/formulaic-1.1.1-py3-none-any.whl
!pip install /kaggle/input/cibmtr-whl-files-for-installation/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/cibmtr-whl-files-for-installation/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/cibmtr-whl-files-for-installation/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=fff51eeea9ac173a550dece809e08f2af5cae0893f8df08756dfd92be2216919
  Stored in directory: /root/.cache/pip/wheels/67/ef/d5/018f452b36cbe7104781a86008d5dd66a1793afbd8864af8bf
Successfully built autograd-gamma
Processing /kaggle/input/cibmtr-whl-files-for-installation/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/cibmtr-whl-files-for-installation/formulaic-1.1.1-py3-none-any.whl
Processing /kaggle/input/cibmtr-whl-files-for-installation/lifelines-0.30.0-py3-none-any.whl


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from sklearn.model_selection import train_test_split


# read_data

In [4]:
test=pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")


In [5]:
data=pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
data.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223


In [6]:
data['age_comorbidity_interaction'] = data['age_at_hct'] * data['comorbidity_score']
test['age_comorbidity_interaction'] = test['age_at_hct'] * test['comorbidity_score']
baseline=2008
data["years_since_baseline"]=data["year_hct"]-baseline
data["years_since_baseline"]
test["years_since_baseline"]=test["year_hct"]-baseline
test["years_since_baseline"]


0     8
1     0
2    11
Name: years_since_baseline, dtype: int64

In [7]:
cat_data=data.select_dtypes(["object"])
number_data=data.select_dtypes(["int64","float64"])
cat_test=test.select_dtypes(["object"])
number_test=test.select_dtypes(["int64","float64"])

# feature engineering

# Imputer

In [8]:
cat_imputer=SimpleImputer(strategy='most_frequent')
number_impute=SimpleImputer(strategy='mean')
number_impute_test=SimpleImputer(strategy='mean')
cat_data = pd.DataFrame(cat_imputer.fit_transform(cat_data), columns=cat_data.columns)
cat_test = pd.DataFrame(cat_imputer.transform(cat_test), columns=cat_test.columns)
number_data = pd.DataFrame(number_impute.fit_transform(number_data), columns=number_data.columns)
number_test = pd.DataFrame(number_impute_test.fit_transform(number_test), columns=number_test.columns)

# normalization

In [9]:
scaler = StandardScaler()
number_data=number_data.drop("ID",axis=1)
number_test=number_test.drop("ID",axis=1)

number_data_scaled=scaler.fit_transform(number_data)
number_test_scaled=scaler.fit_transform(number_test)

In [10]:
number_data_scaled = pd.DataFrame(number_data_scaled, columns=number_data.columns)
number_test_scaled = pd.DataFrame(number_test_scaled, columns=number_test.columns)


# concat

In [11]:
id_data=data["ID"]
id_test=test["ID"]

In [12]:
data=pd.concat([id_data,number_data_scaled,cat_data],axis=1)
data.head()

,ID,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,...,gvhd_proph,rheum_issue,sex_match,race_group,hepatic_mild,tce_div_match,donor_related,melphalan_dose,cardiac,pulm_moderate
0,0,-5.610384e-16,-6.357585e-16,0.753386,0.811944,1.075758e-15,0.649272,0.755021,0.585379,0.661997,...,FKalone,No,M-F,More than one race,No,Permissive mismatched,Unrelated,"N/A, Mel not given",No,No
1,1,5.949953e-01,8.039862e-01,0.753386,0.811944,8.374026e-01,0.649272,0.755021,0.585379,0.661997,...,Other GVHD Prophylaxis,No,F-F,Asian,No,Permissive mismatched,Related,"N/A, Mel not given",No,Yes
2,2,5.949953e-01,8.039862e-01,0.753386,0.811944,8.374026e-01,0.649272,0.755021,0.585379,0.661997,...,Cyclophosphamide alone,No,F-M,More than one race,No,Permissive mismatched,Related,"N/A, Mel not given",No,No
3,3,5.949953e-01,8.039862e-01,0.753386,0.811944,8.374026e-01,0.649272,0.755021,0.585379,0.661997,...,FK+ MMF +- others,No,M-M,White,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",No,No
4,4,5.949953e-01,8.039862e-01,0.753386,0.811944,8.374026e-01,0.649272,-0.144184,0.585379,0.661997,...,TDEPLETION +- other,No,M-F,American Indian or Alaska Native,No,Permissive mismatched,Related,MEL,No,No


In [13]:
test=pd.concat([id_test,number_test_scaled,cat_test],axis=1)
test.head()

,ID,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,...,gvhd_proph,rheum_issue,sex_match,race_group,hepatic_mild,tce_div_match,donor_related,melphalan_dose,cardiac,pulm_moderate
0,28800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,FKalone,No,M-F,More than one race,No,Permissive mismatched,Unrelated,"N/A, Mel not given",No,No
1,28801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Other GVHD Prophylaxis,No,F-F,Asian,No,Permissive mismatched,Related,"N/A, Mel not given",No,Yes
2,28802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Cyclophosphamide alone,No,F-M,More than one race,No,Permissive mismatched,Related,"N/A, Mel not given",No,No


# encoding

In [14]:

def label_encode_datasets(train_df, test_df, categ_fields):
    train_encoded = data.copy()
    test_encoded = test.copy()
    le = LabelEncoder()
    
    for column in cat_data:
        print(f'Encoding: {column} ...')
        le.fit(train_encoded[column])
        
        train_encoded[column] = le.transform(train_encoded[column])
        if column in test_encoded.columns:
            test_encoded[column] = test_encoded[column].map(lambda s: le.transform([s])[0] if s in le.classes_ else None)
            test_encoded[column].fillna(-1, inplace=True)
            test_encoded[column] = test_encoded[column].astype(int)

    return train_encoded, test_encoded



In [15]:

remove_variables = data[['ID', 'efs', 'efs_time']]
features = [feat for feat in data if feat not in remove_variables]
categorical_features = [feat for feat in data[features] if data[feat].dtype == 'object']
numerical_features = [feat for feat in data[features] if feat not in categorical_features]


In [16]:
trn_encoded, tst_encoded = label_encode_datasets(data, test, categorical_features)

Encoding: dri_score ...
Encoding: psych_disturb ...
Encoding: cyto_score ...
Encoding: diabetes ...
Encoding: tbi_status ...
Encoding: arrhythmia ...
Encoding: graft_type ...
Encoding: vent_hist ...
Encoding: renal_issue ...
Encoding: pulm_severe ...
Encoding: prim_disease_hct ...
Encoding: cmv_status ...
Encoding: tce_imm_match ...
Encoding: rituximab ...
Encoding: prod_type ...
Encoding: cyto_score_detail ...
Encoding: conditioning_intensity ...
Encoding: ethnicity ...
Encoding: obesity ...
Encoding: mrd_hct ...
Encoding: in_vivo_tcd ...
Encoding: tce_match ...
Encoding: hepatic_severe ...
Encoding: prior_tumor ...
Encoding: peptic_ulcer ...
Encoding: gvhd_proph ...
Encoding: rheum_issue ...
Encoding: sex_match ...
Encoding: race_group ...
Encoding: hepatic_mild ...
Encoding: tce_div_match ...
Encoding: donor_related ...
Encoding: melphalan_dose ...
Encoding: cardiac ...
Encoding: pulm_moderate ...


# Model

In [17]:
trn_encoded["efs"]=trn_encoded["efs"].abs()

In [18]:
trn_encoded['efs'] = (trn_encoded['efs']).astype(int)

print(trn_encoded['efs'].unique()) 


[1 0]


In [19]:
def train_random_survival_forest(df, time_col, event_col, feature_cols, n_estimators=100, random_state=42):

    survival_data = Surv.from_dataframe(event=event_col, time=time_col, data=df)
    X = df[feature_cols]

    X_train, X_test, y_train, y_test = train_test_split(X, survival_data, test_size=0.3, random_state=random_state)

    model = RandomSurvivalForest(
        n_estimators=n_estimators,
        random_state=random_state,
        n_jobs=-1,
        max_features='sqrt',
        min_samples_leaf=3
    )
    model.fit(X_train, y_train)

    score = model.score(X_test, y_test)
    print('Score: ', score)

    return model, X_train, X_test, y_train, y_test

In [20]:
time_col = "efs_time"
event_col = "efs"
feature_cols = categorical_features + numerical_features

model, X_train, X_test, y_train, y_test = train_random_survival_forest(trn_encoded, 
                                                                       time_col, 
                                                                       event_col, 
                                                                       feature_cols, 
                                                                       n_estimators=12)

Score:  0.8063339432223295


In [21]:

predictions_train = model.predict(trn_encoded[feature_cols])

predictions_train



array([ 6853.5160194 ,  2063.98888889, 16485.22175735, ...,
        8135.27777778,  7360.79166667, 10422.8375    ])

In [22]:

predictions_test = model.predict(tst_encoded[feature_cols])

predictions_test



array([ 6441.59702381,  3048.10138889, 12768.09543651])

In [23]:
from sksurv.metrics import concordance_index_censored
predictions = model.predict(X_test)
c_index = concordance_index_censored(
        y_test["efs"], y_test["efs_time"], predictions
    )[0]
print(f"Model Score (Internal): {model.score(X_test, y_test)}")
print(f"C-index (Test Data): {c_index}")


Model Score (Internal): 0.8063339432223295
C-index (Test Data): 0.8063339432223295


# sub

In [24]:
predictions_test = predictions_test[:len(test['ID'])]


In [25]:
# Save Test Predictions
test_results = pd.DataFrame({
    'ID': test['ID'],  
    'prediction': predictions_test
})

test_results.to_csv('submission.csv', index=False)

In [26]:
test_results

,ID,prediction
0,28800,6441.597024
1,28801,3048.101389
2,28802,12768.095437
